# Tried to explore different way to launch a separate ipython session so that if it crashes the original jupyter kernel doesn't crash.
	- This would be useful for the visualizations

2023-08-28 - Not working anywhere
`poetry add jupyter_client zmq`

In [ ]:
import subprocess

def launch_new_ipython_instance():
    subprocess.Popen(["ipython"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Launch the new IPython instance
launch_new_ipython_instance()


In [ ]:
from attrs import define, field, Factory
from jupyter_client import KernelManager
import uuid

@define(slots=False)
class IPythonKernelInstance:
    def __init__(self):
        self.kernel_manager = KernelManager()
        self.kernel_manager.start_kernel()
        self.kernel_client = self.kernel_manager.client()
        self.kernel_client.start_channels()
        
    def execute(self, code: str):
        msg_id = uuid.uuid4().hex
        self.kernel_client.execute(code, msg_id)
        return self._get_execute_reply(msg_id)
        
    def _get_execute_reply(self, msg_id):
        while True:
            msg = self.kernel_client.get_iopub_msg(timeout=1)
            if msg['msg_type'] == 'execute_result' and msg['parent_header']['msg_id'] == msg_id:
                return msg['content']['data']

# Create new IPython Kernel Instance
ipy_instance = IPythonKernelInstance()

# Execute code and get result
result = ipy_instance.execute("2 + 2")
print("Result:", result['text/plain'])


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
c = get_config()

NameError: name 'get_config' is not defined